In [0]:
import pandas as pd
import numpy as np

In [0]:
data = pd.read_csv("급식식단 데이터.csv", encoding='utf-8-sig', engine="python",index_col=[0])

# 1. 알레르기 정보 컬럼으로 나누기

In [0]:
import re
from tqdm import tqdm_notebook

col_name = [str(i) for i in range(1,19)]

for i in tqdm_notebook(range(len(data))):
    s = data.loc[i,'요리명']
    allergy = re.findall('\d+', s) # 문자열에서 숫자만 빼오기
    allergy = list(set(allergy)) # 리스트 값 중복 제거
    for  num in col_name:
        if num in allergy:
            data.loc[i, num] = 1
        else:
            data.loc[i, num] = 0

In [0]:
data.rename(columns = {"1":"1.난류","2":"2.우유","3":"3.메밀","4":"4.땅콩","5":"5.대두","6":"6.밀","7":"7.고등어","8":"8.게",
                      "9":"9.새우","10":"10.돼지고기","11":"11.복숭아","12":"12.토마토","13":"13.아황산염","14":"14.호두",
                       "15":"15.닭고기","16":"16.쇠고기","17":"17.오징어","18":"18.조개류(굴,전복,홍합 등)"}, inplace = True)

In [0]:
data.to_csv("foodAlle.csv",encoding="utf-8-sig")

In [0]:
data=pd.read_csv("foodAlle.csv", encoding='utf-8-sig', engine="python",index_col=[0])

# 2. 요리 이름 문자열 변환

In [0]:
import re
l=[]
for i in range(len(data["요리명"])):
    a=re.sub("\d|\.|-??|\([^)]*\)","",data["요리명"][i])
    a=a.replace("<br/>",",").replace("*","").replace("★","").replace("#","").replace("!","").replace("^","")
    a=a.replace("H","").replace("h","")
    b=re.sub(r'\([^)]*\)', '', a)
    b=b.replace("()","").replace("\\","").replace("@","").replace("*","").replace("^","")
    l.append(b)
data["요리 이름"]=l

# 3-1. 영양정보 컬럼으로 각각 나눠서 저장(단위도 함계)

In [0]:
nutrient=['']*len(data)

for n in data.index:
    nutrient[n]=data['영양정보'][n]
    nutrient[n]=nutrient[n].replace('탄수화물(g) :','').replace('단백질(g) :','').replace('지방(g) :','').replace('티아민(mg) :','').replace('비타민A(R.E) :','').replace('리보플라빈(mg) :','').replace('비타민C(mg) :','').replace('칼슘(mg) :','').replace('철분(mg) :','')

In [0]:
total=[]
for i in range(len(nutrient)):
    sen=nutrient[i]
    total.append(sen.split('<br/>')) #<br/> 기준으로 나누기
    
for i in range(len(total)):
    for j in range(len(total[0])):
        total[i][j]=total[i][j].strip() #공백제거
        total[i][j]=float(total[i][j]) #형변환 

all_list=[[],[],[],[],[],[],[],[],[]]

for i in range(len(total)):
    for j in range(9):
        all_list[j].append(total[i][j])

In [0]:
#딕셔너리 형태로 저장 
nutrient_split={'탄수화물(g)':all_list[0],'단백질(g)':all_list[1],'지방(g)':all_list[2],'비타민A(mg)':all_list[3],'티아민(mg)':all_list[4],'리보플라빈(mg)':all_list[5],'비타민C(mg)':all_list[6],'칼슘(mg)':all_list[7],'철분(mg)':all_list[8]}

In [0]:
data2=pd.DataFrame(nutrient_split)

#중복정보 삭제
data=data.drop(columns=['영양정보'])

#data 합치기 
result=pd.concat([data,data2],axis=1)

# 3-2. 칼로리 단위 제거, 칼럼에 단위 포함

In [0]:
cal=[]
for i in range(len(result["칼로리정보"])):
    cal.append(float(result["칼로리정보"][i].split(" ")[0]))
result["칼로리(Kcal)"]=cal

In [0]:
result=result.drop(columns=["요리명","칼로리정보"])

In [0]:
# 열의 순서 바꾸기
result=result[['학교명', '식사코드', '식사명', '급식일자','요리 이름', '칼로리(Kcal)' ,'1.난류', '2.우유', '3.메밀',
               '4.땅콩', '5.대두','6.밀', '7.고등어', '8.게', '9.새우', '10.돼지고기', '11.복숭아', '12.토마토', '13.아황산염',
       '14.호두', '15.닭고기', '16.쇠고기', '17.오징어', '18.조개류(굴,전복,홍합 등)', '탄수화물(g)',
       '단백질(g)', '지방(g)', '비타민A(mg)', '티아민(mg)', '리보플라빈(mg)', '비타민C(mg)',
       '칼슘(mg)', '철분(mg)']]

# 4-1. 조식, 중식, 석식 모두 있는 고등학교 찾기

In [0]:
#중복된 학교명들 받아오기
school=['']*len(result)

for n in result.index:
    school[n]=result['학교명'][n]
    
# 중복된 식사명들 받아오기
eat_time=['']*len(result)

for n in result.index:
    eat_time[n]=result['식사명'][n]

In [0]:
#조식인 index추출
one_time=[]
for i in range(len(eat_time)):
    if '조식' in eat_time[i]:
        one_time.append(i) 
        
# 조식을 먹는 학교만 찾아낸다. 이름이 중복
one_eat=[]
for i in range(len(one_time)):
    one_eat.append(school[one_time[i]])
    
# 학교이름 중복 없애기
one_eat_unique=[]

for i in one_eat:
    if i not in one_eat_unique:
        one_eat_unique.append(i)

#중식도 동일하게 
two_time=[]

for i in range(len(eat_time)):
    if '중식' in eat_time[i]:
        two_time.append(i)
        
# 중식을 먹는 학교만 찾아낸다.
two_eat=[]
for i in range(len(two_time)):
    two_eat.append(school[two_time[i]])
    
two_eat_unique=[]

for i in two_eat:
    if i not in two_eat_unique:
        two_eat_unique.append(i)

#석식
three_time=[]
for i in range(len(eat_time)):
    if '석식' in eat_time[i]:
        three_time.append(i)
        
# 석식을 먹는 학교만 찾아낸다.
three_eat=[]
for i in range(len(three_time)):
    three_eat.append(school[three_time[i]])
    
three_eat_unique=[]

for i in three_eat:
    if i not in three_eat_unique:
        three_eat_unique.append(i)

In [0]:
#조식 중식 석식 다 먹는 학교만 골라내기
all_eat=[]
for i in one_eat_unique:
    if i in two_eat_unique and three_eat_unique:
        all_eat.append(i)

In [0]:
dic={'삼시세끼':all_eat}
df=pd.DataFrame(dic)

In [0]:
df.to_csv("삼식고등학교.csv",encoding='utf-8-sig')

# 4-2. 삼식이

In [0]:
df=pd.read_csv("삼식고등학교.csv",encoding='utf-8-sig',index_col=[0],  engine='python')

In [0]:
row=[]
for i in range(len(df)):
    for j in range(len(result)):
        if result.iloc[j,0]==df.iloc[i,0]:
            row.append(result.loc[j].tolist())

In [0]:
d=pd.DataFrame(row,columns=result.columns)

In [0]:
# 열 순서 바꾸기
d2=d[['학교명', '급식일자','식사코드','요리 이름', '칼로리(Kcal)',  '1.난류', '2.우유', '3.메밀',
       '4.땅콩', '5.대두', '6.밀', '7.고등어', '8.게', '9.새우', '10.돼지고기', '11.복숭아',
       '12.토마토', '13.아황산염', '14.호두', '15.닭고기', '16.쇠고기', '17.오징어',
       '18.조개류(굴,전복,홍합 등)', '탄수화물(g)', '단백질(g)', '지방(g)', '비타민A(mg)',
       '티아민(mg)', '리보플라빈(mg)', '비타민C(mg)', '칼슘(mg)', '철분(mg)' ]]

In [0]:
# type 바꾸기
d2=d2.astype({'급식일자':int,'식사코드':int, '칼로리(Kcal)':np.float, '탄수화물(g)':np.float, '단백질(g)':np.float, '지방(g)':np.float,
              '비타민A(mg)':np.float, '티아민(mg)':np.float, '리보플라빈(mg)':np.float, '비타민C(mg)':np.float, '칼슘(mg)':np.float,
              '철분(mg)':np.float,'1.난류':np.float, '2.우유':np.float, '3.메밀':np.float, '4.땅콩':np.float, '5.대두':np.float,
              '6.밀':np.float, '7.고등어':np.float, '8.게':np.float, '9.새우':np.float, '10.돼지고기':np.float, '11.복숭아':np.float,
              '12.토마토':np.float, '13.아황산염':np.float, '14.호두':np.float, '15.닭고기':np.float, '16.쇠고기':np.float, '17.오징어':np.float,
              '18.조개류(굴,전복,홍합 등)':np.float})

nd=pd.DataFrame(0,columns=['급식일자', '식사코드', '칼로리(Kcal)',  '1.난류', '2.우유', '3.메밀','4.땅콩', '5.대두',
                           '6.밀', '7.고등어', '8.게', '9.새우', '10.돼지고기', '11.복숭아', '12.토마토', '13.아황산염', 
                           '14.호두', '15.닭고기', '16.쇠고기', '17.오징어', '18.조개류(굴,전복,홍합 등)', '탄수화물(g)', 
                           '단백질(g)', '지방(g)', '비타민A(mg)', '티아민(mg)', '리보플라빈(mg)', '비타민C(mg)', 
                           '칼슘(mg)', '철분(mg)', '식단'],index=range(1))

In [0]:
for hn in df["삼시세끼"]:
    s=d2[d2["학교명"]==hn]
    
    # 요리명 제외하고 나머지 열 더하기
    s2=s[['급식일자', '식사코드', '칼로리(Kcal)',  '1.난류', '2.우유', '3.메밀','4.땅콩', '5.대두',
          '6.밀', '7.고등어', '8.게', '9.새우', '10.돼지고기', '11.복숭아', '12.토마토', '13.아황산염', 
          '14.호두', '15.닭고기', '16.쇠고기', '17.오징어', '18.조개류(굴,전복,홍합 등)', '탄수화물(g)', 
          '단백질(g)', '지방(g)', '비타민A(mg)', '티아민(mg)', '리보플라빈(mg)', '비타민C(mg)', 
          '칼슘(mg)', '철분(mg)']]
    
    group=s2.groupby(s2["급식일자"]).sum()
    row2=[]
    idx=[]
    for i in range(len(group)):
        if group.iloc[i,0]==6:
            idx.append(group.index[i])
            row2.append(group.iloc[i].tolist())
    
    group2=pd.DataFrame(row2,columns=group.columns,index=idx)

    
    # 요리명 합치기
    totaln=[]
    for j in range(len(idx)):
        n=[]
        for k in range(3):
            n.append(s[s2["급식일자"]==idx[j]].iloc[k,3])
        totaln.append("//".join(n))
    
    # 학교와 요리명, 그외의 정보 합치기
    g=group2.reset_index()   
    g["식단"]=totaln
    
    # index를 급식일자로 바꾸기
    g.rename(columns= {"index": "급식일자"}, inplace = True) 
    nd=pd.concat([nd,g])

In [0]:
nd=nd.reset_index(drop=True)
# 열 순서 바꾸기
nd=nd[['식단', '칼로리(Kcal)', '1.난류', '2.우유', '3.메밀', '4.땅콩', '5.대두',
       '6.밀', '7.고등어', '8.게', '9.새우', '10.돼지고기', '11.복숭아', '12.토마토', '13.아황산염',
       '14.호두', '15.닭고기', '16.쇠고기', '17.오징어', '18.조개류(굴,전복,홍합 등)', '탄수화물(g)',
       '단백질(g)', '지방(g)', '비타민A(mg)', '티아민(mg)', '리보플라빈(mg)', '비타민C(mg)',
       '칼슘(mg)', '철분(mg)']]
nd=nd[1:]

In [0]:
nd.to_csv("삼식이.csv",encoding='utf-8-sig')

# 4-3. 일이식이

In [0]:
#중복된 학교명들 받아오기
schoolname=result["학교명"].unique().tolist()
threeschool=df["삼시세끼"].tolist()
onetwo=[]
for i in range(len(schoolname)):
    if schoolname[i] not in threeschool:
        onetwo.append(schoolname[i]) 

In [0]:
row2=[]
for j in range(len(result)):
    if result.iloc[j,0] in onetwo:
        row2.append(result.loc[j].tolist())

In [0]:
d3=pd.DataFrame(row2,columns=result.columns)

In [0]:
# 열 순서 바꾸기
d4=d3[['학교명', '급식일자','식사코드','요리 이름', '칼로리(Kcal)',  '1.난류', '2.우유', '3.메밀',
       '4.땅콩', '5.대두', '6.밀', '7.고등어', '8.게', '9.새우', '10.돼지고기', '11.복숭아',
       '12.토마토', '13.아황산염', '14.호두', '15.닭고기', '16.쇠고기', '17.오징어',
       '18.조개류(굴,전복,홍합 등)', '탄수화물(g)', '단백질(g)', '지방(g)', '비타민A(mg)',
       '티아민(mg)', '리보플라빈(mg)', '비타민C(mg)', '칼슘(mg)', '철분(mg)' ]]

In [0]:
# type 바꾸기
d4=d4.astype({'급식일자':int,'식사코드':int, '칼로리(Kcal)':np.float, '탄수화물(g)':np.float, '단백질(g)':np.float, '지방(g)':np.float,
              '비타민A(mg)':np.float, '티아민(mg)':np.float, '리보플라빈(mg)':np.float, '비타민C(mg)':np.float, '칼슘(mg)':np.float,
              '철분(mg)':np.float,'1.난류':np.float, '2.우유':np.float, '3.메밀':np.float, '4.땅콩':np.float, '5.대두':np.float,
              '6.밀':np.float, '7.고등어':np.float, '8.게':np.float, '9.새우':np.float, '10.돼지고기':np.float, '11.복숭아':np.float,
              '12.토마토':np.float, '13.아황산염':np.float, '14.호두':np.float, '15.닭고기':np.float, '16.쇠고기':np.float, '17.오징어':np.float,
              '18.조개류(굴,전복,홍합 등)':np.float})

nd2=pd.DataFrame(0,columns=['급식일자', '식사코드', '칼로리(Kcal)',  '1.난류', '2.우유', '3.메밀','4.땅콩', '5.대두',
                           '6.밀', '7.고등어', '8.게', '9.새우', '10.돼지고기', '11.복숭아', '12.토마토', '13.아황산염', 
                           '14.호두', '15.닭고기', '16.쇠고기', '17.오징어', '18.조개류(굴,전복,홍합 등)', '탄수화물(g)', 
                           '단백질(g)', '지방(g)', '비타민A(mg)', '티아민(mg)', '리보플라빈(mg)', '비타민C(mg)', 
                           '칼슘(mg)', '철분(mg)', '식단'],index=range(1))
nd3=pd.DataFrame(0,columns=['급식일자', '식사코드', '칼로리(Kcal)',  '1.난류', '2.우유', '3.메밀','4.땅콩', '5.대두',
                           '6.밀', '7.고등어', '8.게', '9.새우', '10.돼지고기', '11.복숭아', '12.토마토', '13.아황산염', 
                           '14.호두', '15.닭고기', '16.쇠고기', '17.오징어', '18.조개류(굴,전복,홍합 등)', '탄수화물(g)', 
                           '단백질(g)', '지방(g)', '비타민A(mg)', '티아민(mg)', '리보플라빈(mg)', '비타민C(mg)', 
                           '칼슘(mg)', '철분(mg)', '식단'],index=range(1))

삼식고등학교 이외의 고등학교

In [0]:
for hn in onetwo:
    s=d4[d4["학교명"]==hn]
    
    # 요리명 제외하고 나머지 열 더하기
    s2=s[['급식일자', '식사코드', '칼로리(Kcal)',  '1.난류', '2.우유', '3.메밀','4.땅콩', '5.대두',
          '6.밀', '7.고등어', '8.게', '9.새우', '10.돼지고기', '11.복숭아', '12.토마토', '13.아황산염', 
          '14.호두', '15.닭고기', '16.쇠고기', '17.오징어', '18.조개류(굴,전복,홍합 등)', '탄수화물(g)', 
          '단백질(g)', '지방(g)', '비타민A(mg)', '티아민(mg)', '리보플라빈(mg)', '비타민C(mg)', 
          '칼슘(mg)', '철분(mg)']]
    
    group=s2.groupby(s2["급식일자"]).sum()
    row4=[]
    idx=[]
    for i in range(len(group)):
        idx.append(group.index[i]) # 날짜
        row4.append(group.iloc[i].tolist())
    
    group2=pd.DataFrame(row4,columns=group.columns,index=idx)
    
    # 요리명 합치기
    totaln=[]
    for j in range(len(idx)):
        n=[]
        b=s[s2["급식일자"]==idx[j]]
        for k in range(len(b)):
            n.append(b.iloc[k,3])
        totaln.append("//".join(n))
    
    # 학교와 요리명, 그외의 정보 합치기
    g=group2.reset_index()   
    g["식단"]=totaln
    
    # index를 급식일자로 바꾸기
    g.rename(columns= {"index": "급식일자"}, inplace = True) 
    nd2=pd.concat([nd2,g])

삼식고등학교에서 삼식이 아닌 것

In [0]:
for hn in df["삼시세끼"]:
    s=d2[d2["학교명"]==hn]
    
    # 요리명 제외하고 나머지 열 더하기
    s2=s[['급식일자', '식사코드', '칼로리(Kcal)',  '1.난류', '2.우유', '3.메밀','4.땅콩', '5.대두',
          '6.밀', '7.고등어', '8.게', '9.새우', '10.돼지고기', '11.복숭아', '12.토마토', '13.아황산염', 
          '14.호두', '15.닭고기', '16.쇠고기', '17.오징어', '18.조개류(굴,전복,홍합 등)', '탄수화물(g)', 
          '단백질(g)', '지방(g)', '비타민A(mg)', '티아민(mg)', '리보플라빈(mg)', '비타민C(mg)', 
          '칼슘(mg)', '철분(mg)']]
    
    group=s2.groupby(s2["급식일자"]).sum()
    row2=[]
    idx=[]
    for i in range(len(group)):
        if group.iloc[i,0]!=6:
            idx.append(group.index[i])
            row2.append(group.iloc[i].tolist())
    
    group2=pd.DataFrame(row2,columns=group.columns,index=idx)

    # 요리명 합치기
    totaln2=[]
    for j in range(len(idx)):
        n=[]
        b=s[s2["급식일자"]==idx[j]]
        for k in range(len(b)):
            n.append(b.iloc[k,3])
        totaln2.append("//".join(n))
    
    # 학교와 요리명, 그외의 정보 합치기
    g=group2.reset_index()   
    g["식단"]=totaln2
    
    # index를 급식일자로 바꾸기
    g.rename(columns= {"index": "급식일자"}, inplace = True) 
    nd3=pd.concat([nd3,g])

In [0]:
nd2=nd2.reset_index(drop=True)
nd3=nd3.reset_index(drop=True)

In [0]:
# 열 순서 바꾸기
nd2=nd2[['식단', '칼로리(Kcal)', '1.난류', '2.우유', '3.메밀', '4.땅콩', '5.대두',
       '6.밀', '7.고등어', '8.게', '9.새우', '10.돼지고기', '11.복숭아', '12.토마토', '13.아황산염',
       '14.호두', '15.닭고기', '16.쇠고기', '17.오징어', '18.조개류(굴,전복,홍합 등)', '탄수화물(g)',
       '단백질(g)', '지방(g)', '비타민A(mg)', '티아민(mg)', '리보플라빈(mg)', '비타민C(mg)',
       '칼슘(mg)', '철분(mg)']]
nd2=nd2[1:] # 1행 삭제
# 열 순서 바꾸기
nd3=nd3[['식단', '칼로리(Kcal)', '1.난류', '2.우유', '3.메밀', '4.땅콩', '5.대두',
       '6.밀', '7.고등어', '8.게', '9.새우', '10.돼지고기', '11.복숭아', '12.토마토', '13.아황산염',
       '14.호두', '15.닭고기', '16.쇠고기', '17.오징어', '18.조개류(굴,전복,홍합 등)', '탄수화물(g)',
       '단백질(g)', '지방(g)', '비타민A(mg)', '티아민(mg)', '리보플라빈(mg)', '비타민C(mg)',
       '칼슘(mg)', '철분(mg)']]
nd3=nd3[1:] # 1행 삭제

In [0]:
nd4=pd.concat([nd2,nd3])
nd4=nd4.reset_index(drop=True)
nd4.to_csv("일이식이.csv",encoding='utf-8-sig')